<a href="https://colab.research.google.com/github/MoralesTorres/ML_Notebooks/blob/master/API_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install fastapi uvicorn sqlalchemy pydantic python-multipart pyngrok


In [4]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


In [5]:
import threading, uvicorn

def run():
    uvicorn.run("app:app", host="0.0.0.0", port=8000, log_level="info")

threading.Thread(target=run, daemon=True).start()
print("✅ Uvicorn running on port 8000")


✅ Uvicorn running on port 8000


testing the port


In [6]:
import subprocess, re, time

p = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

public_url = None
for _ in range(60):
    line = p.stdout.readline()
    if line:
        m = re.search(r"(https://[-\w]+\.trycloudflare\.com)", line)
        if m:
            public_url = m.group(1)
            break
    time.sleep(0.2)

print("✅ Public URL:", public_url)
print("Swagger:", public_url + "/docs")


✅ Public URL: https://constitute-troy-dispatched-person.trycloudflare.com
Swagger: https://constitute-troy-dispatched-person.trycloudflare.com/docs


In [7]:
%%writefile app.py
import os
import csv
import shutil
from datetime import datetime
from typing import List

from fastapi import FastAPI, UploadFile, File, Depends, HTTPException
from pydantic import BaseModel, Field
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, DeclarativeBase, Session
from sqlalchemy.exc import IntegrityError

# -----------------------------
# DB (SQLite local in Colab VM)
# -----------------------------
DATABASE_URL = "sqlite:///./challenge.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autocommit=False, autoflush=False)

class Base(DeclarativeBase):
    pass

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# -----------------------------
# Models
# -----------------------------
class Department(Base):
    __tablename__ = "departments"
    id = Column(Integer, primary_key=True)
    department = Column(String, nullable=False)

class Job(Base):
    __tablename__ = "jobs"
    id = Column(Integer, primary_key=True)
    job = Column(String, nullable=False)

class Employee(Base):
    __tablename__ = "hired_employees"
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    datetime = Column(DateTime, nullable=False)
    department_id = Column(Integer, ForeignKey("departments.id"), nullable=False)
    job_id = Column(Integer, ForeignKey("jobs.id"), nullable=False)

Base.metadata.create_all(bind=engine)

# -----------------------------
# Schemas (Batch 1..1000)
# -----------------------------
MAX_BATCH = 1000

class DepartmentCreate(BaseModel):
    id: int = Field(..., ge=1)
    department: str

class JobCreate(BaseModel):
    id: int = Field(..., ge=1)
    job: str

class EmployeeCreate(BaseModel):
    id: int = Field(..., ge=1)
    name: str
    datetime: datetime
    department_id: int = Field(..., ge=1)
    job_id: int = Field(..., ge=1)

def validate_batch(n: int):
    if n < 1 or n > MAX_BATCH:
        raise HTTPException(status_code=400, detail=f"Batch must be 1..{MAX_BATCH}. Got {n}")

# -----------------------------
# CSV parsing (NO HEADERS)
# -----------------------------
def parse_departments_csv(path: str):
    items = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        for row in reader:
            if len(row) < 2:
                continue
            items.append({"id": int(row[0]), "department": row[1].strip()})
    return items

def parse_jobs_csv(path: str):
    items = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        for row in reader:
            if len(row) < 2:
                continue
            items.append({"id": int(row[0]), "job": row[1].strip()})
    return items

def parse_hired_employees_csv(path: str):
    items = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        for row in reader:
            if len(row) < 5:
                continue
            raw_dt = row[2].strip().replace("Z", "")
            try:
                dt = datetime.fromisoformat(raw_dt)
            except ValueError:
                raise HTTPException(status_code=400, detail=f"Invalid datetime: {row[2]}")
            items.append({
                "id": int(row[0]),
                "name": row[1].strip(),
                "datetime": dt,
                "department_id": int(row[3]),
                "job_id": int(row[4]),
            })
    return items

# -----------------------------
# FastAPI
# -----------------------------
UPLOAD_DIR = "./uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

app = FastAPI(title="API Challenge - DB Migration (Colab)")

def save_upload(file: UploadFile) -> str:
    path = os.path.join(UPLOAD_DIR, file.filename)
    with open(path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)
    return path

@app.get("/health")
def health():
    return {"status": "ok"}

# ---- Upload endpoints (CSV -> DB)
@app.post("/upload/departments")
def upload_departments(file: UploadFile = File(...), db: Session = Depends(get_db)):
    path = save_upload(file)
    items = parse_departments_csv(path)
    validate_batch(len(items))
    try:
        db.add_all([Department(**x) for x in items])
        db.commit()
        return {"table": "departments", "inserted": len(items)}
    except IntegrityError as e:
        db.rollback()
        raise HTTPException(status_code=409, detail=str(e.orig))

@app.post("/upload/jobs")
def upload_jobs(file: UploadFile = File(...), db: Session = Depends(get_db)):
    path = save_upload(file)
    items = parse_jobs_csv(path)
    validate_batch(len(items))
    try:
        db.add_all([Job(**x) for x in items])
        db.commit()
        return {"table": "jobs", "inserted": len(items)}
    except IntegrityError as e:
        db.rollback()
        raise HTTPException(status_code=409, detail=str(e.orig))

@app.post("/upload/hired_employees")
def upload_hired_employees(file: UploadFile = File(...), db: Session = Depends(get_db)):
    path = save_upload(file)
    items = parse_hired_employees_csv(path)
    validate_batch(len(items))
    try:
        db.add_all([Employee(**x) for x in items])
        db.commit()
        return {"table": "hired_employees", "inserted": len(items)}
    except IntegrityError as e:
        db.rollback()
        raise HTTPException(status_code=409, detail=str(e.orig))

# ---- Batch JSON endpoints (1..1000)
@app.post("/batch/departments")
def batch_departments(payload: List[DepartmentCreate], db: Session = Depends(get_db)):
    validate_batch(len(payload))
    try:
        db.add_all([Department(**x.model_dump()) for x in payload])
        db.commit()
        return {"table": "departments", "inserted": len(payload)}
    except IntegrityError as e:
        db.rollback()
        raise HTTPException(status_code=409, detail=str(e.orig))

@app.post("/batch/jobs")
def batch_jobs(payload: List[JobCreate], db: Session = Depends(get_db)):
    validate_batch(len(payload))
    try:
        db.add_all([Job(**x.model_dump()) for x in payload])
        db.commit()
        return {"table": "jobs", "inserted": len(payload)}
    except IntegrityError as e:
        db.rollback()
        raise HTTPException(status_code=409, detail=str(e.orig))

@app.post("/batch/hired_employees")
def batch_hired_employees(payload: List[EmployeeCreate], db: Session = Depends(get_db)):
    validate_batch(len(payload))
    try:
        db.add_all([Employee(**x.model_dump()) for x in payload])
        db.commit()
        return {"table": "hired_employees", "inserted": len(payload)}
    except IntegrityError as e:
        db.rollback()
        raise HTTPException(status_code=409, detail=str(e.orig))


Overwriting app.py


cloud connection further